In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [6]:
data_dir = '/kaggle/input/dermnet/train'
#making windows path
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('/kaggle/input/dermnet/train')

In [7]:
#counting number of images
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

15557


In [8]:
#making labels
labels = ['Acne and Rosacea Photos',
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
 'Atopic Dermatitis Photos',
 'Bullous Disease Photos',
 'Cellulitis Impetigo and other Bacterial Infections',
 'Eczema Photos',
 'Exanthems and Drug Eruptions',
 'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation',
 'Lupus and other Connective Tissue diseases',
 'Melanoma Skin Cancer Nevi and Moles',
 'Nail Fungus and other Nail Disease',
 'Poison Ivy Photos and other Contact Dermatitis',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Scabies Lyme Disease and other Infestations and Bites',
 'Seborrheic Keratoses and other Benign Tumors',
 'Systemic Disease',
 'Tinea Ringworm Candidiasis and other Fungal Infections',
  'Urticaria Hives',
 'Vascular Tumors',
 'Vasculitis Photos',
 'Warts Molluscum and other Viral Infections']

In [9]:
#making dictionary 
images_dict = {}
for i in labels:
  images_dict[i] = list(data_dir.glob(i + '/*'))

In [11]:
#making dictionary of labels
num = 0
labels_dict = {}
for i in labels:
  labels_dict[i] = num
  num += 1

In [13]:
#resizing images
X, y = [], []

for name, images in images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(180,180))
        X.append(resized_img)
        y.append(labels_dict[name])

In [14]:
#making numpy array
X = np.array(X)
y = np.array(y)

In [15]:
#splitting data set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [17]:
#scaling images
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [18]:
#data augmentation to improve accuracy
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(180, 180,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.experimental.preprocessing.RandomContrast(0.1),
  ]
)

In [19]:
#building cnn
num_classes = 23

model = Sequential([
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'), #2d convulational layer
  layers.MaxPooling2D(), #maxpooling
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2), #dropping random nodes
  layers.Flatten(),
  layers.Dense(128, activation='relu'), #dense layer
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(X_train_scaled, y_train, epochs=20) 

Epoch 1/20
365/365 [==============================] - 305s 816ms/step - loss: 2.8970 - accuracy: 0.1352
Epoch 2/20
365/365 [==============================] - 294s 806ms/step - loss: 2.7838 - accuracy: 0.1719
Epoch 3/20
365/365 [==============================] - 298s 816ms/step - loss: 2.7055 - accuracy: 0.1975
Epoch 4/20
365/365 [==============================] - 294s 806ms/step - loss: 2.6341 - accuracy: 0.2255
Epoch 5/20
365/365 [==============================] - 291s 797ms/step - loss: 2.5753 - accuracy: 0.2295
Epoch 6/20
365/365 [==============================] - 290s 795ms/step - loss: 2.5235 - accuracy: 0.2480
Epoch 7/20
365/365 [==============================] - 297s 813ms/step - loss: 2.4726 - accuracy: 0.2640
Epoch 8/20
365/365 [==============================] - 292s 799ms/step - loss: 2.4363 - accuracy: 0.2711
Epoch 9/20
365/365 [==============================] - 290s 796ms/step - loss: 2.3910 - accuracy: 0.2827
Epoch 10/20
365/365 [==============================] - 294s 805m

In [20]:
model.evaluate(X_test_scaled,y_test)

122/122 [==============================] - 24s 191ms/step - loss: 2.7478 - accuracy: 0.2563


[2.747849941253662, 0.2562982141971588]